Importing Words:

In [2]:
import random
from itertools import permutations


In [3]:

scrabble_words = []

with open('words.txt') as wordsFile:
  for word in wordsFile:
    scrabble_words.append(word.strip('\n').upper())

print(scrabble_words[-100:-10])

# print(words)

['ZYGOCACTUS', 'ZYGOCACTUSES', 'ZYGOCARDIAC', 'ZYGODACTYL', 'ZYGODACTYLIC', 'ZYGODACTYLISM', 'ZYGODACTYLISMS', 'ZYGODACTYLOUS', 'ZYGODACTYLS', 'ZYGODONT', 'ZYGOID', 'ZYGOMA', 'ZYGOMAS', 'ZYGOMATA', 'ZYGOMATIC', 'ZYGOMATICS', 'ZYGOMORPHIC', 'ZYGOMORPHIES', 'ZYGOMORPHISM', 'ZYGOMORPHISMS', 'ZYGOMORPHOUS', 'ZYGOMORPHY', 'ZYGOMYCETE', 'ZYGOMYCETES', 'ZYGOMYCETOUS', 'ZYGON', 'ZYGOPHYLLACEOUS', 'ZYGOPHYTE', 'ZYGOPHYTES', 'ZYGOPLEURAL', 'ZYGOSE', 'ZYGOSES', 'ZYGOSIS', 'ZYGOSITIES', 'ZYGOSITY', 'ZYGOSPERM', 'ZYGOSPERMS', 'ZYGOSPHENE', 'ZYGOSPHENES', 'ZYGOSPORE', 'ZYGOSPORES', 'ZYGOSPORIC', 'ZYGOTE', 'ZYGOTENE', 'ZYGOTENES', 'ZYGOTES', 'ZYGOTIC', 'ZYGOTICALLY', 'ZYLONITE', 'ZYLONITES', 'ZYMASE', 'ZYMASES', 'ZYME', 'ZYMES', 'ZYMIC', 'ZYMITE', 'ZYMITES', 'ZYMOGEN', 'ZYMOGENE', 'ZYMOGENES', 'ZYMOGENESES', 'ZYMOGENESIS', 'ZYMOGENIC', 'ZYMOGENS', 'ZYMOGRAM', 'ZYMOGRAMS', 'ZYMOID', 'ZYMOLOGIC', 'ZYMOLOGICAL', 'ZYMOLOGIES', 'ZYMOLOGIST', 'ZYMOLOGISTS', 'ZYMOLOGY', 'ZYMOLYSES', 'ZYMOLYSIS', 'ZYMOLYTIC'

## Brute Force Method

In [7]:


# Replace with the appropriate letter values in your Scrabble set
# LETTER_VALUES = {
#     'A': 1, 'B': 3, 'C': 3, 'D': 2, 'E': 1, 'F': 4, 'G': 2, 'H': 4,
#     'I': 1, 'J': 8, 'K': 5, 'L': 1, 'M': 3, 'N': 1, 'O': 1, 'P': 3,
#     'Q': 10, 'R': 1, 'S': 1, 'T': 1, 'U': 1, 'V': 4, 'W': 4, 'X': 8,
#     'Y': 4, 'Z': 10
# }

# def calculate_score(word):
#     return sum(LETTER_VALUES[letter] for letter in word.upper())

# def get_valid_words(letters):
#     valid_words = set()
#     for i in range(1, len(letters) + 1):
#         for perm in permutations(letters, i):
#             word = ''.join(perm)
#             if word.upper() in [word.upper() for word in words.words()]:
#                 valid_words.add(word)
#     return valid_words

# def get_best_scrabble_words(letters):
#     valid_words = get_valid_words(letters)
#     scored_words = [(word, calculate_score(word)) for word in valid_words]
#     sorted_words = sorted(scored_words, key=lambda x: x[1], reverse=True)
#     return sorted_words

# # Example usage
# rack_letters = 'EATOPL'
# best_words = get_best_scrabble_words(rack_letters)
# print(len(best_words))
# print("Best Scrabble words:", best_words[:10])  # Displaying the top 10 words

In [21]:
print(len(words.words()))

236736


In [4]:
class TrieNode:
    def __init__(self, letter, score, is_end_of_word=False, word_score=0, parent_id=None):
        self.letter = letter
        self.score = score
        self.is_end_of_word = is_end_of_word
        self.word_score = word_score
        self.parent_id = parent_id
        self.children = {}

class Trie:
    def __init__(self):
        self.nodes = []
        self.root = self.nodes[self._create_node(None, None, None, False, 0)]

    def _create_node(self, letter, score, parent_id, is_end_of_word, word_score):
        node = TrieNode(letter, score, is_end_of_word, word_score, parent_id)
        self.nodes.append(node)
        return len(self.nodes) - 1  # Return the index of the newly created node

    def insert_word(self, word, scores):
        current_node = self.root

        for i, letter in enumerate(word):
            score = scores[letter]
            is_end_of_word = (i == len(word) - 1)

            if letter not in current_node.children:
                child_id = self._create_node(letter, score, current_node, is_end_of_word, current_node.word_score + score)
                current_node.children[letter] = child_id
                current_node = self.nodes[child_id]
            else:
                child_id = current_node.children[letter]
                child_node = self.nodes[child_id]
                if is_end_of_word:
                    child_node.is_end_of_word = True

                current_node = child_node

    def search_word(self, word):
        current_node = self.root

        for letter in word:
            if letter in current_node.children:
                current_node = self.nodes[current_node.children[letter]]
            else:
                return None  # Word not found

        return current_node
    
    def find_valid_words(self, letters, minimum_word_length=2, must_contain=''):
        valid_words = []
        maximum_score = [0]
        maximum_word = [""]
        

        def dfs(node, current_word, remaining_letters):
            if node.is_end_of_word and len(current_word) >= minimum_word_length:
                valid_words.append({
                    'word': current_word,
                    'score': node.word_score
                })
                if node.word_score > maximum_score[-1]:
                    maximum_score[-1] = node.word_score
                    maximum_word[-1] = current_word

            for letter, child_id in node.children.items():
                child_node = self.nodes[child_id]
                if letter in remaining_letters:
                    new_word = current_word + letter
                    new_remaining_letters = remaining_letters.replace(letter, "", 1)
                    dfs(child_node, new_word, new_remaining_letters)

        current_node = self.root
        for letter in letters:
            # print(letter)
            if letter in current_node.children:
                current_node = self.nodes[self.root.children[letter]]
                dfs(current_node, letter, letters.replace(letter, "", 1))
        
        # filter out words which do not contain the required letters and letters beyond
        if must_contain != '':
            valid_words = [
                {'word': valid_word['word'], 'score': valid_word['score']}
                for valid_word in valid_words
                if all(letter in valid_word['word'] for letter in must_contain)
            ]

        if len(valid_words) > 0:
            valid_words = sorted(valid_words, key=lambda x: x['score'], reverse=True)


        return valid_words, maximum_word[-1], maximum_score[-1]


# Example usage
LETTER_VALUES = {
    'A': 1, 'B': 3, 'C': 3, 'D': 2, 'E': 1, 'F': 4, 'G': 2, 'H': 4,
    'I': 1, 'J': 8, 'K': 5, 'L': 1, 'M': 3, 'N': 1, 'O': 1, 'P': 3,
    'Q': 10, 'R': 1, 'S': 1, 'T': 1, 'U': 1, 'V': 4, 'W': 4, 'X': 8,
    'Y': 4, 'Z': 10
}

trie = Trie()

# Insert words into the trie
# words_to_insert = ['CAT', 'DOG', 'COT', 'DOT', 'COG', 'DOGS']
words_to_insert = scrabble_words
for word in words_to_insert:
    trie.insert_word(word, LETTER_VALUES)

print(trie.root.children)
# # Search for a word in the trie
# search_word = 'DOG'
# result_node = trie.search_word(search_word)

# if result_node:
#     print(f"The word '{search_word}' is valid in Scrabble.")
#     print(f"Score for the word: {result_node.word_score}")
# else:
#     print(f"The word '{search_word}' is not valid in Scrabble.")

given_letters = 'CAISPRHV'
valid_words, maximum_word, maximum_score  = trie.find_valid_words(given_letters)

# Display results
for word in valid_words:
    print(f"The word {word['word']} is valid in Scrabble with a score of {word['score']}.")

print(len(valid_words))

print(f"The maximum score is {maximum_score}, using the word {maximum_word}.")


{'A': 1, 'B': 37042, 'C': 68564, 'D': 121673, 'E': 157414, 'F': 182749, 'G': 204923, 'H': 224213, 'I': 247594, 'J': 271049, 'K': 275686, 'L': 282518, 'M': 299287, 'N': 333814, 'O': 350964, 'P': 371135, 'Q': 424843, 'R': 427775, 'S': 459071, 'T': 526493, 'U': 557328, 'V': 584905, 'W': 594632, 'X': 606742, 'Y': 607518, 'Z': 609469}
The word CAPISH is valid in Scrabble with a score of 13.
The word CHAVS is valid in Scrabble with a score of 13.
The word CHIRPS is valid in Scrabble with a score of 13.
The word CHIVS is valid in Scrabble with a score of 13.
The word SCARPH is valid in Scrabble with a score of 13.
The word SCHAV is valid in Scrabble with a score of 13.
The word PHASIC is valid in Scrabble with a score of 13.
The word CAPHS is valid in Scrabble with a score of 12.
The word CHAPS is valid in Scrabble with a score of 12.
The word CHAV is valid in Scrabble with a score of 12.
The word CHIPS is valid in Scrabble with a score of 12.
The word CHIRP is valid in Scrabble with a score 

In [6]:
class ScrabbleGame:
    def __init__(self):
        # Define the initial set of letters in the pool
        self.letter_pool = {
            'A': 9, 'B': 2, 'C': 2, 'D': 4, 'E': 12, 'F': 2, 'G': 3, 'H': 2,
            'I': 9, 'J': 1, 'K': 1, 'L': 4, 'M': 2, 'N': 6, 'O': 8, 'P': 2,
            'Q': 1, 'R': 6, 'S': 4, 'T': 6, 'U': 4, 'V': 2, 'W': 2, 'X': 1,
            'Y': 2, 'Z': 1
        }

        self.word_bank = [""]

        self.player_pool = []
        self.continue_game = True

    def display_rack(self):
        print("Your current rack:", self.player_pool)

    def display_pool(self):
        print("Letters remaining in the pool:")
        print(self.letter_pool)
        # for letter, count in self.letter_pool.items():
        #     print(f"{letter}: {count}")
    
    def display_word_bank(self):
        print("Words played so far:")
        print(self.word_bank)

    def calculate_score(_self, word):
        return sum(LETTER_VALUES[letter] for letter in word.upper())

    def output_score(self):
        print("Your current score:", sum(self.calculate_score(word) for word in self.word_bank))
    
    def find_optimal_words_to_play(self):
        playable_options = []
        for word in self.word_bank:
            print("CHECKING WORD: ", word + ''.join(self.player_pool))
            valid_words, maximum_word, maximum_score  = trie.find_valid_words(word + ''.join(self.player_pool), max(3, len(word) + 1), word)
            for valid_word in valid_words:
                playable_options.append({
                    "word": valid_word['word'],
                    "turn_score": valid_word['score'],
                    "formed_from": word
                })
        
        playable_options = sorted(playable_options, key=lambda x: x["turn_score"], reverse=True)

        print("The optimal words to play are:")
        print(playable_options)
        return playable_options
        
            

    def draw_letters(self, num_letters):
        if (len(self.letter_pool) == 0):
            print("Error: No letters left in the pool.")
            self.continue_game = False
            return []
        if num_letters > len(self.letter_pool):
            print("Error: Not enough letters in the pool.")
            return []

        drawn_letters = random.choices(list(self.letter_pool.keys()), weights=(self.letter_pool.values()), k=num_letters)
        for letter in drawn_letters:
            self.player_pool.append(letter)
            self.letter_pool[letter] -= 1
            if self.letter_pool[letter] == 0:
                del self.letter_pool[letter]

        return drawn_letters

    def play_word(self, word, formed_from=''):
        if word == "":
            print("Skipping turn.")
            return True
        if word.upper() == "!EXIT":
            self.continue_game = False
            return True
        if formed_from not in self.word_bank and formed_from != '':
            print(f"Error: '{formed_from}' is not in your word bank.")
            return False
        

        word_upper = word.upper()
        letters_to_remove = []
        word_to_remove = formed_from
        for letter in word_upper:

            if letter not in self.player_pool and letter not in formed_from:
                print(f"Error: '{letter}' is not in your rack.")
                return False
            if letter in formed_from:
                formed_from = formed_from.replace(letter, '', 1)
            elif letter not in self.player_pool:
                print(f"Error: '{letter}' is not in your rack.")
                return False
            else:
                letters_to_remove.append(letter)

        if formed_from != '':
            print(f"Error: Letters from '{word_to_remove}' not in '{word}'. Missing letters: '{formed_from}'")
            return False
        if len(letters_to_remove) == 0:
            print(f"Error: '{word}' must contain at least one new letter.")
            return False
        
        for letter in letters_to_remove:
            self.player_pool.remove(letter)
        if word_to_remove != '':
            self.word_bank.remove(word_to_remove)
        self.word_bank.append(word_upper)
        print(f"Success! '{word_upper}' played.")
        return True


# Example usage:
scrabble_game = ScrabbleGame()

scrabble_game.draw_letters(2)
while scrabble_game.continue_game:
    scrabble_game.draw_letters(1)
    scrabble_game.display_rack()
    scrabble_game.display_pool()
    scrabble_game.display_word_bank()
    scrabble_game.find_optimal_words_to_play()
    successfully_executed_turn = False
    while not successfully_executed_turn:
        word_to_play = input("Enter a word to play: ")
        formed_from = input("Enter the word formed from (leave blank if word is entirely from pool): ")
        successfully_executed_turn = scrabble_game.play_word(word_to_play.upper(), formed_from.upper())
    scrabble_game.output_score()


# Draw 7 letters for the player's initial rack
# scrabble_game.draw_letters(7)

# # Display the initial rack and letter pool
# scrabble_game.display_rack()
# scrabble_game.display_pool()

# # Play a word
# word_to_play = input("Enter a word to play: ")
# scrabble_game.play_word(word_to_play)

# # Display the updated rack and letter pool
# scrabble_game.display_rack()
# scrabble_game.display_pool()

Your current rack: ['F', 'E', 'I']
Letters remaining in the pool:
{'A': 9, 'B': 2, 'C': 2, 'D': 4, 'E': 11, 'F': 1, 'G': 3, 'H': 2, 'I': 8, 'J': 1, 'K': 1, 'L': 4, 'M': 2, 'N': 6, 'O': 8, 'P': 2, 'Q': 1, 'R': 6, 'S': 4, 'T': 6, 'U': 4, 'V': 2, 'W': 2, 'X': 1, 'Y': 2, 'Z': 1}
Words played so far:
['']
CHECKING WORD:  FEI
The optimal words to play are:
[{'word': 'FIE', 'turn_score': 6, 'formed_from': ''}]
Success! 'FIE' played.
Your current rack: ['S']
Letters remaining in the pool:
{'A': 9, 'B': 2, 'C': 2, 'D': 4, 'E': 11, 'F': 1, 'G': 3, 'H': 2, 'I': 8, 'J': 1, 'K': 1, 'L': 4, 'M': 2, 'N': 6, 'O': 8, 'P': 2, 'Q': 1, 'R': 6, 'S': 3, 'T': 6, 'U': 4, 'V': 2, 'W': 2, 'X': 1, 'Y': 2, 'Z': 1}
Words played so far:
['', 'FIE']
CHECKING WORD:  S
CHECKING WORD:  FIES
The optimal words to play are:
[{'word': 'FEIS', 'turn_score': 7, 'formed_from': 'FIE'}, {'word': 'SEIF', 'turn_score': 7, 'formed_from': 'FIE'}]
Success! 'FIES' played.
Your current rack: ['N']
Letters remaining in the pool:
{'A': 